In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


In [17]:
data = pd.read_csv('../raw_data/zfdata0.csv')
data

,X,Y,ANGLE,CUM_angle,TAIL_P1,TAIL_P2,TAIL_P3,TAIL_P4,TAIL_P5,TAIL_P6,...,L_EYE,R_EYE,Timestamp,STIM_type,SIZE_dot,DIST_dot,AMPL_rot,SPEED_rot,LUM_dot,BGLUM
0,652.2355,369.7303,5.315731,-5.315731,0,-0.105654,-0.011476,0.015549,-0.024984,0.015548,...,-0.115261,0.239720,983,0,0.004,0.1,0.523599,0.35,1,0
1,652.1792,369.8040,5.315613,-5.315613,0,-0.106579,-0.025909,0.028131,-0.025909,0.014625,...,-0.112420,0.237397,988,0,0.004,0.1,0.523599,0.35,1,0
2,652.2500,369.6550,5.314946,-5.314946,0,-0.106319,-0.012141,0.014884,-0.012141,0.014884,...,-0.115186,0.235710,994,0,0.004,0.1,0.523599,0.35,1,0
3,652.2166,369.7576,5.315727,-5.315727,0,-0.107471,-0.026800,0.013732,-0.013292,0.013732,...,-0.117052,0.234698,999,0,0.004,0.1,0.523599,0.35,1,0
4,652.2130,369.7354,5.315329,-5.315330,0,-0.101495,-0.007319,0.046713,-0.007319,0.019707,...,-0.105362,0.248317,4,0,0.004,0.1,0.523599,0.35,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7367,579.7074,645.5211,1.452244,-7.735429,0,-0.011805,0.028728,-0.052249,0.001703,0.028729,...,-0.150881,0.264506,291,4,0.004,0.1,1.047198,0.35,1,0
7368,579.7284,645.6978,1.452245,-7.735430,0,-0.000418,0.040115,-0.040906,-0.000418,0.026608,...,-0.146326,0.256485,310,4,0.004,0.1,1.047198,0.35,1,0
7369,579.7714,646.0289,1.449146,-7.732332,0,-0.019285,0.034756,-0.032782,0.021248,-0.005776,...,-0.152566,0.253526,329,4,0.004,0.1,1.047198,0.35,1,0
7370,579.8110,646.0666,1.448264,-7.731449,0,0.030303,0.030304,0.003278,-0.023727,0.003278,...,-0.147943,0.261025,348,4,0.004,0.1,1.047198,0.35,1,0


In [25]:
data.loc[0,'STIM_type']

0

In [20]:
data.STIM_type.unique()

array([0, 2, 3, 4])

In [18]:
data.AMPL_rot.unique()

array([0.5235989, 1.047198 ])

In [21]:
data[(data['STIM_type']==4) & (data['AMPL_rot']==0.5235989)]

,X,Y,ANGLE,CUM_angle,TAIL_P1,TAIL_P2,TAIL_P3,TAIL_P4,TAIL_P5,TAIL_P6,...,L_EYE,R_EYE,Timestamp,STIM_type,SIZE_dot,DIST_dot,AMPL_rot,SPEED_rot,LUM_dot,BGLUM
1023,660.3515,366.0472,5.791057,-5.791057,0,-0.103075,0.018260,0.004752,-0.035782,-0.049279,...,-0.197517,0.217314,94,4,0.004,0.1,0.523599,0.35,1,0
1024,660.5438,365.9606,5.813534,-5.813534,0,-0.071773,0.009206,-0.017820,-0.017821,-0.031328,...,-0.196328,0.213453,99,4,0.004,0.1,0.523599,0.35,1,0
1025,660.4575,366.0341,5.792017,-5.792017,0,-0.130926,-0.009724,-0.009724,-0.063755,-0.036749,...,-0.211862,0.201888,104,4,0.004,0.1,0.523599,0.35,1,0
1026,660.0733,366.2150,5.791471,-5.791471,0,-0.094274,-0.000096,-0.000096,-0.000096,-0.067488,...,-0.194487,0.209883,109,4,0.004,0.1,0.523599,0.35,1,0
1027,660.4100,366.0542,5.791899,-5.791899,0,-0.071653,0.009324,0.009324,-0.031208,-0.031209,...,-0.196934,0.221099,113,4,0.004,0.1,0.523599,0.35,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6138,520.2013,618.5087,3.516743,-3.516743,0,0.037633,0.024135,0.024135,0.064579,0.024135,...,-0.338667,0.063001,340,4,0.004,0.1,0.523599,0.35,1,0
6139,520.1155,618.4973,3.516217,-3.516217,0,0.038843,0.038844,0.025345,0.079230,0.038843,...,-0.334440,0.067370,359,4,0.004,0.1,0.523599,0.35,1,0
6140,520.0627,618.4786,3.516169,-3.516169,0,0.079944,0.066479,0.025992,0.079944,0.052997,...,-0.316873,0.081489,378,4,0.004,0.1,0.523599,0.35,1,0
6141,519.8740,618.4437,3.524623,-3.524623,0,0.034793,0.021296,0.021295,0.061740,0.034793,...,-0.330151,0.066135,396,4,0.004,0.1,0.523599,0.35,1,0


In [13]:
fig = px.line(data,y=['L_EYE','R_EYE'])
fig